In [1]:
import random
import math

DELTA = 0.01   # Mutation step size
NumEval = 0    # Total number of evaluations

#problem/Convex.txt
#problem/Ackley.txt
#problem/Griewank.txt
def main():
    # Create an instance of numerical optimization problem
    p = createProblem()   # 'p': (expr, domain)
    # Call the search algorithm
    solution, minimum = steepestAscent(p)
    # Show the problem and algorithm settings
    describeProblem(p)
    displaySetting()
    # Report results
    displayResult(solution, minimum)


def createProblem(): ###
    ## Read in an expression and its domain from a file.
    
    ## Then, return a problem 'p'.
    ## 'p' is a tuple of 'expression' and 'domain'.
    ## 'expression' is a string.
    ## 'domain' is a list of 'varNames', 'low', and 'up'.
    ## 'varNames' is a list of variable names.
    ## 'low' is a list of lower bounds of the varaibles.
    ## 'up' is a list of upper bounds of the varaibles.
    fileName = input("Enter the filename of a function")
    infile = open(fileName,"r")
    expression = infile.readline()
    varName =[]
    low=[]
    up=[]
    line = infile.readline()
    while line !="":
        data = line.split(",")
        varName.append(data[0])
        low.append(float(data[1]))  # Convert to float
        up.append(float(data[2]))  # Convert to float
        line = infile.readline().strip()
    infile.close()
    domain = [varName,low,up]    
    
    return expression, domain


def steepestAscent(p):
    current = randomInit(p) # 'current' is a list of values
    valueC = evaluate(current, p)
    while True:
        neighbors = mutants(current, p)
        successor, valueS = bestOf(neighbors, p)
        if valueS >= valueC:
            break
        else:
            current = successor
            valueC = valueS
    return current, valueC


def randomInit(p): ###
    domain = p[1]
    low = domain[1]
    up = domain[2]
    init=[]
    # for i in range(len(low)):
    #     r = random.uniform(low[i],up[i])
    #     init.append(r)
    init = [random.uniform(l,u) for l,u in zip(low,up)]
    return init    # Return a random initial point
                   # as a list of values

def evaluate(current, p):
    ## Evaluate the expression of 'p' after assigning
    ## the values of 'current' to the variables
    global NumEval #글로벌 변수를 계속 바꾸고 싶을때 표시해줘야 한다.
    
    NumEval += 1
    expr = p[0]         # p[0] is function expression
    varNames = p[1][0]  # p[1] is domain
    for i in range(len(varNames)):
        assignment = varNames[i] + '=' + str(current[i])
        exec(assignment)
    return eval(expr)

def mutants(current, p):
    neighbors = []
  
    for i in range(len(current)):
        mutant=mutate(current, i, DELTA,p)  # Adjust the function call
        neighbors.append(mutant)
        mutant =mutate(current,i,-DELTA,p)
        neighbors.append(mutant)
    return neighbors   # Return a set of successors


def mutate(current, i, d, p): ## Mutate i-th of 'current' if legal
    curCopy = current[:]
    domain = p[1]        # [VarNames, low, up]
    l = domain[1][i]     # Lower bound of i-th
    u = domain[2][i]     # Upper bound of i-th
    if l <= (curCopy[i] + d) <= u:
        curCopy[i] += d
    return curCopy

def bestOf(neighbors, p): ###
     # Return the best successor and its value
    best = neighbors[0]
    bestValue = evaluate(best, p)
    for i in neighbors[1:]:
        successorValue = evaluate(i, p)
        if successorValue < bestValue:
            best = i
            bestValue = successorValue
    return best, bestValue

def describeProblem(p):
    print()
    print("Objective function:")
    print(p[0])   # Expression
    print("Search space:")
    varNames = p[1][0] # p[1] is domain: [VarNames, low, up]
    low = p[1][1]
    up = p[1][2]
    for i in range(len(low)):
        print(" " + varNames[i] + ":", (low[i], up[i])) 

def displaySetting():
    print()
    print("Search algorithm: Steepest-Ascent Hill Climbing")
    print()
    print("Mutation step size:", DELTA)

def displayResult(solution, minimum):
    print()
    print("Solution found:")
    print(coordinate(solution))  # Convert list to tuple
    print("Minimum value: {0:,.3f}".format(minimum))
    print()
    print("Total number of evaluations: {0:,}".format(NumEval))

def coordinate(solution):
    c = [round(value, 3) for value in solution]
    return tuple(c)  # Convert the list to a tuple


main()



Objective function:
(x1 - 2) ** 2 +5 * (x2 - 5) ** 2 + 8 * (x3 + 8) ** 2 + 3 * (x4 + 1) ** 2 + 6 * (x5 - 7) ** 2

Search space:
 x1: (-30.0, 30.0)
 x2: (-30.0, 30.0)
 x3: (-30.0, 30.0)
 x4: (-30.0, 30.0)
 x5: (-30.0, 30.0)

Search algorithm: Steepest-Ascent Hill Climbing

Mutation step size: 0.01

Solution found:
(2.004, 4.996, -8.0, -0.996, 6.998)
Minimum value: 0.000

Total number of evaluations: 75,691


먼저, 주어진 코드는 Python으로 구현된 Steepest-Ascent Hill Climbing 알고리즘을 사용하여 수치 최적화 문제를 해결하는 프로그램입니다. 코드를 자세히 설명해드리겠습니다.

createProblem() 함수:

사용자로부터 파일 이름을 입력받아 해당 파일로부터 수식(expression)과 변수의 도메인(domain)을 읽어옵니다.
읽어온 데이터를 이용하여 수식과 도메인으로 이루어진 튜플 'p'를 생성하고 반환합니다.
steepestAscent() 함수:

랜덤한 초기 점(current)을 생성하고, 초기 점의 값(valueC)을 평가합니다.
이후에는 무한 루프를 돌면서 이웃 점(neighbors)을 생성하고, 최적의 후계자(successor)를 찾습니다.
만약 후계자의 값(valueS)이 현재 점의 값(valueC)보다 크거나 같다면 루프를 종료하고 최적의 점과 그 값인 최소값(minimum)을 반환합니다.
randomInit() 함수:

도메인(domain)에서 변수의 하한(low)과 상한(up)을 가져와서 각 변수에 대해 랜덤한 값을 생성합니다.
생성된 값을 초기 점으로 사용하기 위해 리스트로 반환합니다.
evaluate() 함수:

'p'에 포함된 수식(expression)에 현재 값(current)을 변수에 할당한 후, 수식을 평가하여 결과를 반환합니다.
exec 함수를 사용하여 변수 할당을 수행합니다.
전역 변수인 NumEval을 증가시킴으로써 함수 호출 횟수를 계산합니다.
mutants() 함수:

현재 점(current)을 기반으로 이웃 점(neighbors)을 생성합니다.
각 변수에 대해 변이(mutate)를 수행하여 이웃 점을 생성하고, neighbors 리스트에 추가합니다.
변이를 통해 생성된 두 개의 이웃 점을 neighbors 리스트에 추가하는데, 이는 이웃을 더 다양하게 만들기 위한 전략입니다.
mutate() 함수:

주어진 변이(step size)를 사용하여 i번째 변수를 변이시킵니다.
현재 값(current)을 복사한 후, 변이가 도메인의 하한과 상한 내에 있는 경우에만 값을 변이시킵니다.
변이된 값을 반환합니다.
bestOf() 함수:

주어진 이웃 점들(neighbors) 중에서 가장 좋은 후계자(successor)와 해당 후계자의 값(value)을 찾습니다.
각 이웃 점에 대해 평가하여 가장 작은 값을 가진 후계자를 선택합니다.
describeProblem() 함수:

주어진 문제의 목적 함수와 탐색 공간에 대한 정보를 출력합니다.
수식(expression)과 도메인(domain) 정보를 이용하여 출력합니다.
displaySetting() 함수:

프로그램의 설정과 관련된 정보를 출력합니다.
현재 사용 중인 탐색 알고리즘과 변이(step size)에 대한 정보를 출력합니다.
displayResult() 함수:

찾은 해(solution)와 해당 해의 최소값(minimum)을 출력합니다.
좌표 형태로 출력하기 위해 해를 튜플로 변환하여 출력합니다.
NumEval 변수에 저장된 함수 호출 횟수를 출력합니다.

In [4]:
sorce_problem = open("./problem/Convex.txt","r",encoding="utf-8")
sorce_problem

<_io.TextIOWrapper name='./problem/Convex.txt' mode='r' encoding='utf-8'>